# Machine Learning 101 in Python
### Led by Matteo Hessel, UCL ML Alumni, currently a Research Engineer at Google DeepMind, and Bojan Vujatovic, also UCL ML Alumni

[Video recording available here](https://youtu.be/GAusrpSVVPw?list=PLBYyZ2a9bVqyN34JPCzEinA_cXT2ckrCa)

1. Clone [this repository](https://github.com/bojanvujatovic/unsupervised-learning-workshop)
```
git clone https://github.com/bojanvujatovic/unsupervised-learning-workshop.git
cd unsupervised-learning-workshop    
```

2. Check if you have pip installed
```
pip --version  
```
If not, [download](https://bootstrap.pypa.io/get-pip.py) and run with python:
```
curl https://bootstrap.pypa.io/get-pip.py   (for Mac/Linux)
python get-pip.py 
``` 

3. Install all the dependecies necessary for the project (stored in [`requirements.txt`](https://github.com/bojanvujatovic/unsupervised-learning-workshop/blob/master/requirements.txt)) on your machine in the cloned repo (if it fails  `pip install --upgrade pip`)
```
pip install -r requirements.txt
```

---

# Unsupervised Learning

Experience is provided as a stream of multidimensional data which may come in various forms: textual, visual, audio, etc…

**The goal is to find in the data some hidden structure, that can help us explain, compress or recode the data itself.**

### Focus on:
- Clustering (Explain, e.g. through a “generative” story)
- Feature Extraction (Recode, e.g. creating new attributes)

## Clustering
We group the data samples in such a way that elements in the same group are more similar to each other than to those in other groups.

The key element is: what does *similar* mean? How do we *measure* similarity?

- Clusters may focus on some external property (they look similar)
- Clusters may also identify some latent factor, such as all elements being generated by some common sub-process (a “generative story”)

### K-Means

#### Idea
Partition all data in k clusters, identified by a center. Clusters selected to minimize the sum of squared distances between each point and the center of the corresponding cluster.

$$ \operatorname{arg\,min}_s \sum\limits_{i=1}^k \sum\limits_{x \in S_i} \|x - \mu_i\|^2 $$

In order to minimize the sum of squared distances each data point is assigned to the cluster whose center is nearest.
How do we find the ideal locations of the centers?

### Algorithm

Randomly pick $K$ initial cluster centers.

**do**:

       Assign each element to the cluster that has the closest center.
       Recompute the centers as baricenter of the elements of the cluster: 
$$ m_i^{new} = \frac{1}{N_i} \sum\limits_{n \in N_i} x^n $$

**while** (centers are different from previous iteration)

#### How can we meaningfully select K?
![image](img/meaningful.png)

#### What happens with non-spherical clusters?
![image](img/nonsphere.png)
Already with supervised learning I stressed that there is no unique solution to these kind of problems. There is no one algorithm you can always use. With clustering this may even more true.

Finding good clusters is about understanding your data, using the results of your algorithms to guide your search.

If you find that everytime you run your algorithm you get totally different results, that might be suggesting that you are not effectivelt capturing the underlying structure. 

Plotting and visualizing the boundaries of your clusters also may help.


#### What happens with heterogeneous densities?
![image](img/heterogeneous.png)

---

### Practical K-Means 1: Blob Clustering
- artificially creating blobs (clusters) of different type
- running K-Means to try to discover them
- [check out and experiment](https://github.com/bojanvujatovic/unsupervised-learning-workshop/blob/master/blobs_kmeans.py)
```
python blobs_kmeans.py
```
![image](img/blob.png)

---

### Practical K-Means 2: Colour Quantisation

- complete the script [`colour_quatisation_exercise.py`](https://github.com/bojanvujatovic/unsupervised-learning-workshop/blob/master/colour_quatisation_exercise.py). (if you get stuck, there is [`colour_quatisation_solutions.py`](https://github.com/bojanvujatovic/unsupervised-learning-workshop/blob/master/colour_quatisation_solutions.py))

#### Fit KMeans with sample from image colours (faster)

- run clustering on the colours (pixels) of the image
- replace each pixel with its centroid
- try different values of k, what do you notice?
```
python color_quatisation.py
```

--- 

### K-Means: Other Limitations

- Does not manage naturally discrete variables
- No principled way of dealing with missing variables

Sensitive to outliers:

- All data points are assigned to some cluster
- All data points contribute to compute the new centers

Run multiple experiments:

- Algorithm may converge to local optima
- Result depends on initialization

Normalize attributes:

- Not scale invariant

---
### Spectral Clustering

#### Idea
Spectral clustering provides one way to deal with non spherical clusters. The idea is based on a probabilistic model analogous to Page Rank.

A cluster is a group of closely *connected* samples of arbitrary shape

- Connectivity is defined as a function of the distance between samples
- Connectivity is propagated so that far distant samples connected by a sequence of tightly connected samples result closely connected

This is a classic dataset that is very difficult to cluster meaningfully using k means:
![img](img/spectral1.png)

The vectors for points on the same connected part of the data will tend to become identical. The more you look far in the future, and will all have very high values for points in the same connected part, and very low probabilities for those in the other connected component. So in this case you will get basically two groups of vectors, with vectors within each group being very similar and in a almost spherical way. This then becomes the ideal setting for applying k-means.
![img](img/spectral2.png)

#### Algorithm:

Define the similarity matrix A as function of the distances $ A_{i,j} = e^{- \lambda \| x^i - x^j \|^2} $

Define the transition matrix P as a function on similarities $ p(i|j) = \frac{A_{i,j}}{\sum\nolimits_{i} A{i,j}} $

For each data point $x$ :

      compute an approximation of the stationary distribution: 
$$ \phi_k(x) = P^k one-hot (x) $$

The vectors $ \phi_k(x) $ will arrange in very tight semi-spherical clusters
Identify these clusters using k-means

![img](img/power.png)

This is a general and powerful idea. If the problem is difficult, it might be because of how you look at it.
If you take your data and you transform it suitably, Then the problem might become quite easy.

In this case we take datapoints which are aranged in a weird shape and we map each point to a new vector in such a way that, regardless of the shape of the clusters, data points which belong to tightly connected groups will be mapped to tight and approximately spherical clusters the kind of problem we now very well how to solve. Just using k-means!

The same idea may apply in countless problems, not only clustering.
An entire subfield of unsupervised learning, called feature extraction, focuses exactly on this finding representations of your data that make it easier to solve other problems.

Among the problems that can benefit from this there are all supervised learning tasks.
The most common approach to any SL task will indeed usually include a preprocessing step in which feature extraction algorithm are used to transform the data into a new representation where classification or regression are easier


---
## Feature Extraction

Feature Extraction is the subfield of unsupervised learning that deals with building better representations of your data.

A feature extraction algorithm takes as input a set of vectors, and returns a mapping transforming the original data points to new vectors.

$$ x = (x_1, x_1, \ldots , x_1) \in R^n  $$
$$Extract : D \rightarrow \phi(x)$$
$$ dim(\phi(x)) < dim(x) $$

#### Vectors are important here!
A vector is an ordered sequence of elements. 

Each location in the sequence measures how much the vectors extends along one specific direction.
The set of directions, each associated with a location in your vector forms what is called a Reference System.

![img](img/vector.png)

## PCA

### Idea
PCA  =  [*Principal Component Analysis*](http://setosa.io/ev/principal-component-analysis/)

PCA selects a new coordinate system of smaller dimensionality

The new attributes of the resulting vectors are linear combinations of the original attributes


The new reference system is selected in order to preserve as much information as possible: minimize reconstruction error. 
This is done maximizing the variability of data along each direction. 
It also provides an ordering of the resulting directions. 
We can then remove the directions that are less important in the sense that the different datapoints differ very little in their values.

#### Example 1
![img](img/eg1.png)

#### Example 2
![img](img/eg2.png)


### Reconstruction Error
PCA gives you an explicit way of computing a new representation. It also provides a way of reconstructin the orginal data from the new representation of reduced dimensionality.


The more aggressively you reduce the representation’s dimensionality the more information is lost, leading to poorer reconstruction. This gives us a nice way of selecting how many directions to use.

![img](img/reconstruction.png)

---

### Practical PCA 1: MNIST: PCA + kNN

- PCA helps supervised tasks
- Reduced overfitting
- 100 optimal numDimensions
- Check out and experiment: [`python mnist_pca_knn.py`](https://github.com/bojanvujatovic/unsupervised-learning-workshop/blob/master/mnist_pca_knn.py)

![img](img/KNN.png)

--- 

### Practical PCA 2: Modify the above

- Modify the experiment
- Improve performance
- Combine PCA with better SL algorithms (e.g. Decision Trees, Random Forests)

---

### Practical PCA 3: Reconstruct

MNIST digits dataset, reducing dimensionality with PCA: [`python mnist_pca_visualise.py`](https://github.com/bojanvujatovic/unsupervised-learning-workshop/blob/master/mnist_pca_visualise.py)

![img](img/reconstruction.png)

- Many correlated features (pixels) in images
- Helps with compression 

---

### Practical PCA 4: Eigenfaces

- complete the script [`eigenfaces_exercise.py`](https://github.com/bojanvujatovic/unsupervised-learning-workshop/blob/master/eigenfaces_exercise.py). (if you get stuck, there is [`eigenfaces_solution.py`](https://github.com/bojanvujatovic/unsupervised-learning-workshop/blob/master/eigenfaces_solutions.py))
- apply PCA to face images
- reshape principal components and plot as images
- can you explain what you see?